In [5]:
# merge_datasets_specific.py
import shutil
from pathlib import Path

root = Path("data")
datasetPath = Path("dataset")
datasets = [
    root / "all-books",
    root / "book",
    root / "Book_200img",
    root / "book_detection2",
    root / "book_1",
    root / "book_2",
    root / "book_3",
    root / "book_4",
    root / "book_5"
]

out = datasetPath / "combined"
imgs_out = out / "images"
lbls_out = out / "labels"

for split in ["train", "val"]:
    (imgs_out / split).mkdir(parents=True, exist_ok=True)
    (lbls_out / split).mkdir(parents=True, exist_ok=True)

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp"}

def copy_pair(src_img, src_lbl, dest_img, dest_lbl):
    dest_img.parent.mkdir(parents=True, exist_ok=True)
    dest_lbl.parent.mkdir(parents=True, exist_ok=True)
    shutil.copy2(src_img, dest_img)
    shutil.copy2(src_lbl, dest_lbl)

summary = {}
for i, ds in enumerate(datasets, start=1):
    prefix = f"ds{i}"
    summary[prefix] = {"copied":0, "missing_label":0, "no_split":0}
    if not ds.exists():
        print(f"[WARN] dataset not found: {ds} — skipping")
        continue

    for split in ["train","val"]:
        imgs_dir = ds / split / "images"
        lbls_dir = ds / split / "labels"
        if not imgs_dir.exists():
            print(f"[INFO] {prefix}: no {split}/images — пропуск {split}")
            summary[prefix]["no_split"] += 1
            continue
        img_files = [p for p in imgs_dir.iterdir() if p.suffix.lower() in IMG_EXTS]
        for img in img_files:
            lbl = lbls_dir / f"{img.stem}.txt"
            if not lbl.exists():
                summary[prefix]["missing_label"] += 1
                continue
            new_img = imgs_out / split / f"{prefix}_{img.name}"
            new_lbl = lbls_out / split / f"{prefix}_{lbl.name}"
            copy_pair(img, lbl, new_img, new_lbl)
            summary[prefix]["copied"] += 1

    print(f"[DONE] {prefix}: copied={summary[prefix]['copied']}, missing_label={summary[prefix]['missing_label']}")

print("=== SUMMARY ===")
for k,v in summary.items():
    print(k, v)
print("Combined folder ready at:", out)


[DONE] ds1: copied=1863, missing_label=0
[INFO] ds2: no val/images — пропускаю val
[DONE] ds2: copied=859, missing_label=0
[DONE] ds3: copied=180, missing_label=0
[DONE] ds4: copied=2094, missing_label=0
[DONE] ds5: copied=894, missing_label=0
[DONE] ds6: copied=1035, missing_label=0
[DONE] ds7: copied=600, missing_label=0
[INFO] ds8: no val/images — пропускаю val
[DONE] ds8: copied=2079, missing_label=0
[INFO] ds9: no val/images — пропускаю val
[DONE] ds9: copied=2025, missing_label=0
=== SUMMARY ===
ds1 {'copied': 1863, 'missing_label': 0, 'no_split': 0}
ds2 {'copied': 859, 'missing_label': 0, 'no_split': 1}
ds3 {'copied': 180, 'missing_label': 0, 'no_split': 0}
ds4 {'copied': 2094, 'missing_label': 0, 'no_split': 0}
ds5 {'copied': 894, 'missing_label': 0, 'no_split': 0}
ds6 {'copied': 1035, 'missing_label': 0, 'no_split': 0}
ds7 {'copied': 600, 'missing_label': 0, 'no_split': 0}
ds8 {'copied': 2079, 'missing_label': 0, 'no_split': 1}
ds9 {'copied': 2025, 'missing_label': 0, 'no_spli

In [6]:
import json
import os
import shutil

coco_path = "data/coco2017"
combined_path = "dataset/combined"

splits = {"train2017": "train", "val2017": "val"}

for coco_split, yolo_split in splits.items():
    ann_file = os.path.join(coco_path, "annotations", f"instances_{coco_split}.json")
    with open(ann_file, "r") as f:
        coco = json.load(f)

    book_id = 84  # id "book" в COCO

    anns_book = [ann for ann in coco["annotations"] if ann["category_id"] == book_id]

    img_ids = set([a["image_id"] for a in anns_book])
    imgs_book = [img for img in coco["images"] if img["id"] in img_ids]

    img_save_dir = os.path.join(combined_path, "images", yolo_split)
    lbl_save_dir = os.path.join(combined_path, "labels", yolo_split)
    os.makedirs(img_save_dir, exist_ok=True)
    os.makedirs(lbl_save_dir, exist_ok=True)

    src_img_dir = os.path.join(coco_path, coco_split)

    for img in imgs_book:
        img_id = img["id"]
        w, h = img["width"], img["height"]

        anns = [a for a in anns_book if a["image_id"] == img_id]

        base_name = f"coco_{img['file_name']}"
        dst_img = os.path.join(img_save_dir, base_name)
        src_img = os.path.join(src_img_dir, img["file_name"])
        shutil.copyfile(src_img, dst_img)

        lines = []
        for a in anns:
            x, y, bw, bh = a["bbox"]
            xc = (x + bw / 2) / w
            yc = (y + bh / 2) / h
            bw /= w
            bh /= h
            lines.append(f"0 {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}")

        label_file = os.path.join(lbl_save_dir, base_name.replace(".jpg", ".txt"))
        with open(label_file, "w") as f:
            f.write("\n".join(lines))

    print(f"{yolo_split}: добавлено {len(imgs_book)} изображений с книгами")


train: добавлено 5332 изображений с книгами
val: добавлено 230 изображений с книгами


In [14]:
# make_combined_data_yaml.py
import yaml
from pathlib import Path

root = Path("dataset")
combined = root / "combined"
data_yaml = combined / "data.yaml"

candidates = list(root.rglob("data.yaml"))
names = ["book"]
for p in candidates:
    try:
        d = yaml.safe_load(p.read_text())
        if isinstance(d, dict) and 'names' in d:
            names = d['names']
            print(f"[INFO] Found names in {p}")
            break
    except Exception:
        continue

if isinstance(names, dict):
    items = sorted(names.items(), key=lambda x: int(x[0]))
    names = [v for k,v in items]

if names is None:
    names = ["book"]  

data = {
    'train': "../images/train",
    'val': "../images/val",
    'names': names
}

data_yaml.parent.mkdir(parents=True, exist_ok=True)
data_yaml.write_text(yaml.safe_dump(data, sort_keys=False, allow_unicode=True))
print("Wrote", data_yaml)
if not names:
    print("WARNING: 'names' is empty")


Wrote dataset\combined\data.yaml
